In [ ]:
import numpy as np
import pandas as pd
from SimulatedAnnealingbis import SimulatedAnnealing as Sim_an2
from SimulatedAnnealingtris import SimulatedAnnealing as Sim_an3

lmbd = 0.5

iterations = 500
cycles = 5
alpha = 0.75

version = 3

In [ ]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

def create_data_model(dataset):
    """Stores the data for the problem."""
    cities = dataset.x
    dist =  np.zeros((dataset.N, dataset.N))
    radius = np.inf
    center = None
    min_dist = np.inf
    min_i = None
    min_j = None
    max_dist = 0
    max_i = None
    max_j = None
    for i in range(dataset.N):
      r = np.linalg.norm(cities[i] - [0.5, 0.5])  # or most populous
      if r < radius:
        radius = r
        center = i
      for j in range(dataset.N):
        dist[i, j] = np.linalg.norm(cities[i] - cities[j])
        if dist[i, j] < min_dist and dist[i, j] != 0:
          min_dist = dist[i, j]
          min_i = i
          min_j = j
        if dist[i, j] > max_dist:
          max_dist = dist[i, j]
          max_i = i
          max_j = j
    print(cities[center])

    data = {}
    data['distance_matrix'] = np.round(dist / min_dist)
    data['num_vehicles'] = 1
    data['depot'] = center
    data['min_dist'] = min_dist
    data['min_i'] = min_i
    data['min_j'] = min_j
    data['max_dist'] = max_dist
    data['max_i'] = max_i
    data['max_j'] = max_j
    return data

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {}'.format(solution.ObjectiveValue() * data['min_dist']))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [ ]:
class Dataset(object):
    def __init__(self, N: int, x: np.ndarray, v: np.ndarray):
        self.N = N
        self.x = x
        self.v = v

def load_csv(path: str) -> Dataset:
    file = pd.read_csv(path)
    file = file.set_index('city id').sort_index()
    return Dataset(len(file), file.loc[['position x','position y']].to_numpy(), file.loc['normalized population'].to_numpy())

def write_csv(path: str, simulated_annealing) -> None:
    sol = pd.DataFrame(columns=['city id', 'bool'])
    for i, assignment in enumerate(simulated_annealing.best_S._assignments):
        sol = sol.append([i+1, assignment])
    sol.to_csv(path, header = False)

In [ ]:
g_competition = load_csv("test.csv")
print(g_competition.N)
print(g_competition.x)
print(g_competition.v)

In [ ]:
if version == 2:
    SA = Sim_an2(lmbd, g_competition, alpha=alpha)
    SA.heat_cool_cycles(iterations, cycles)
elif version == 3:
    # Instantiate the data problem.
    data = create_data_model(g_competition)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], [data['min_i']], [data['min_j']])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    SA = Sim_an3(lmbd, g_competition, alpha=alpha, order=get_routes(solution, routing, manager)[0])
    SA.heat_cool_cycles(iterations, cycles)